# Ungraded Lab: Capstone Project Lab

## 📋 Overview 
Welcome to the Capstone Project Lab! In this comprehensive hands-on session, you'll apply all the SQL concepts you've learned throughout the course to analyze a complex dataset from TechMart, a growing retail chain. You'll clean data, write advanced queries, and produce a well-documented analysis report. This lab simulates real-world data analysis challenges, preparing you for your future career as a data scientist.

## 🎯 Learning Outcomes
By the end of this lab, you will be able to:
- Clean and prepare complex datasets using SQL
- Write advanced SQL queries involving subqueries, CTEs, and window functions
- Perform comprehensive data analysis across multiple related tables
- Leverage generative AI tools to optimize SQL queries and enhance performance
- Produce a well-documented data analysis report


## 📚 Dataset Information
You'll be working with the <b>TechMart</b> dataset, which contains information about a retail chain's operations across North America and Europe. The dataset includes:
- <b>Employee_Records:</b> Information about employees, their roles, and sales performance
- <b>Product_Details:</b> Details about products, including categories and inventory
- <b>Customer_Demographics:</b> Customer information and loyalty program status
- <b>Sales_Transactions:</b> Transaction data linking customers, products, and employees


## 🖥️ Activities

### Activity 1: Data Exploration and Cleaning 

Before diving into analysis, it's crucial to understand and clean our dataset. We'll start by examining each table and addressing any data quality issues.

<b>Step 1:</b> Connect to the database, then load and display tables:

In [1]:
import sqlite3
import pandas as pd

# Setting up the database. DO NOT edit the code given below
from techsmart_db_setup import setup_database
setup_database() 
conn = sqlite3.connect('techsmart.db')

# Load and display tables
tables = ['Employee_Records', 'Product_Details', 'Customer_Demographics', 'Sales_Transactions']
for table in tables:
    query = f"SELECT * FROM {table} LIMIT 5"
    df = pd.read_sql_query(query, conn)
    print(f"\n{table}:")
    display(df)

✅ Database setup complete: Tables created and populated with data!

Employee_Records:


,employee_id,role,store_location,sales_performance
0,1,Cashier,New York,None
1,2,Cashier,Los Angeles,3000
2,3,Supervisor,Phoenix,None
3,4,Manager,Chicago,3000
4,5,Manager,Phoenix,3000



Product_Details:


,product_id,product_name,category,price,stock
0,106,Tablet,Electronics,one hundred,200.0
1,101,Keyboard,Accessories,fifty,100.0
2,108,Keyboard,Accessories,fifty,500.0
3,104,Speaker,Electronics,500,150.0
4,107,Speaker,Electronics,fifty,300.0



Customer_Demographics:


,customer_id,age,gender,location,loyalty_program
0,1,35,F,San Antonio,None
1,2,50,M,London,No
2,3,37,M,Austin,No
3,4,None,M,San Antonio,None
4,5,28,M,San Antonio,None



Sales_Transactions:


,transaction_id,customer_id,product_id,employee_id,quantity,total_amount,sale_date
0,1,13.0,105,2.0,4,90.0,2025-03-03 00:00:00
1,2,37.0,105,51.0,2,90.0,2025-03-08 00:00:00
2,3,5.0,106a,NaN,three,NaN,2025-03-01 00:00:00
3,4,35.0,109b,22.0,three,40.0,2025-03-04 00:00:00
4,5,15.0,101,45.0,2,NaN,2025-03-06 00:00:00


<b>Step 2:</b> Identify and handle missing values:

In [2]:
# Example for Employee_Records
query = """
SELECT COUNT(*) as total_rows,
       SUM(CASE WHEN sales_performance IS NULL OR sales_performance = 'nan' THEN 1 ELSE 0 END) as missing_sales
FROM Employee_Records
"""
df = pd.read_sql_query(query, conn)
display(df)

,total_rows,missing_sales
0,100,11


<b>Step 3: Try it yourself: </b>Write queries to identify missing values in other tables

In [9]:
# Analyzing Product_Details for wrong price entries
query = """
SELECT 
    COUNT(*) AS total_rows,
    SUM(
        CASE 
            WHEN price IS NULL 
                 OR (price NOT GLOB '*[0-9]*')  -- filter for non-numeric text
                 OR CAST(price AS REAL) IS NULL  -- fails for words like 'fifty'
            THEN 1
            ELSE 0
        END
    ) AS wrong_price
FROM Product_Details;
"""
df = pd.read_sql_query(query, conn)
display(df)

,total_rows,wrong_price
0,100,41


<b>Step 4:</b> Clean inconsistent data formats:

In [16]:
# Example: Standardize sales_performance in Employee_Records
query = """
UPDATE Employee_Records
SET sales_performance = CASE
    WHEN sales_performance = 'nan' THEN NULL
    WHEN sales_performance = 'five thousand' THEN '5000'
    ELSE sales_performance
END
"""
cursor = conn.cursor()
cursor.execute(query)
conn.commit()

**Step 5: Try it yourself:** Clean inconsistent data in other tables


In [32]:
query = """
UPDATE Product_Details
SET price = CASE
    WHEN price = 'nan' THEN NULL
    WHEN price = 'fifty' THEN '50'
    WHEN price = 'one hundred' THEN '100'
    ELSE price
END
"""

# Execute the update
conn.execute(query)
conn.commit()  # make sure to commit changes

# Read the table to see the updated data
df = pd.read_sql_query("SELECT * FROM Product_Details", conn)
display(df)

# Now analyze how many are left from these intial 41

# Analyzing Product_Details for wrong price entries
query = """
SELECT 
    COUNT(*) AS total_rows,
    SUM(
        CASE 
            WHEN price IS NULL 
                 OR (price NOT GLOB '*[0-9]*')  -- filter for non-numeric text
                 OR CAST(price AS REAL) IS NULL  -- fails for words like 'fifty'
            THEN 1
            ELSE 0
        END
    ) AS wrong_price
FROM Product_Details;
"""
df = pd.read_sql_query(query, conn)
display(df) #shows 8 left, analyze those

,product_id,product_name,category,price,stock
0,106,Tablet,Electronics,100,200.0
1,101,Keyboard,Accessories,50,100.0
2,108,Keyboard,Accessories,50,500.0
3,104,Speaker,Electronics,500,150.0
4,107,Speaker,Electronics,50,300.0
...,...,...,...,...,...
95,109,Monitor,Accessories,300,50.0
96,101,Camera,Electronics,25,300.0
97,110,Camera,Electronics,500,200.0
98,107,Speaker,Accessories,50,500.0


,total_rows,wrong_price
0,100,8


<b>💡 Tip:</b> Use CASE statements to handle multiple conditions when cleaning data.

### Activity 2: Advanced Data Analysis  

Now that our data is clean, let's perform some advanced analysis to gain insights into TechMart's operations.

<b>Step 1:</b> Analyze employee performance by location:

In [33]:
query = """
WITH emp_sales AS (
    SELECT store_location, 
           AVG(CAST(sales_performance AS FLOAT)) as avg_sales,
           COUNT(*) as employee_count
    FROM Employee_Records
    WHERE sales_performance IS NOT NULL
    GROUP BY store_location
)
SELECT store_location, avg_sales, employee_count,
       RANK() OVER (ORDER BY avg_sales DESC) as location_rank
FROM emp_sales
ORDER BY avg_sales DESC
"""
df = pd.read_sql_query(query, conn)
display(df)

,store_location,avg_sales,employee_count,location_rank
0,Chicago,3875.000000,8,1
1,Phoenix,3312.500000,16,2
2,London,3214.285714,14,3
3,New York,2823.529412,17,4
4,Los Angeles,2692.307692,13,5
5,Paris,2600.000000,10,6
6,Houston,2363.636364,11,7


<b>Step 2:</b> Identify top-selling products by category:

In [34]:
query = """
SELECT *
FROM (
    SELECT p.category,
           p.product_name,
           SUM(s.quantity) AS total_sold,
           RANK() OVER (
               PARTITION BY p.category
               ORDER BY SUM(s.quantity) DESC
           ) AS rank_in_category
    FROM Sales_Transactions s
    JOIN Product_Details p ON s.product_id = p.product_id
    GROUP BY p.category, p.product_name
)
WHERE rank_in_category <= 3
ORDER BY category, total_sold DESC
"""
df = pd.read_sql_query(query, conn)
display(df)

,category,product_name,total_sold,rank_in_category
0,Accessories,Keyboard,156.0,1
1,Accessories,Charger,123.0,2
2,Accessories,Speaker,85.0,3
3,Electronics,Tablet,134.0,1
4,Electronics,Mouse,129.0,2
5,Electronics,Headphones,110.0,3


 <b>Step 3: Try it yourself</b>  Analyze customer purchasing behavior:

In [ ]:
# Your turn: Write a query to analyze customer purchasing behavior
# Hint: Join Customer_Demographics with Sales_Transactions and use window functions

query = """
<YOUR CODE HERE>
"""
df = pd.read_sql_query(query, conn)
display(df)

### Activity 3: Performance Optimization 
As our dataset grows, query performance becomes crucial. Let's optimize some of our complex queries. The below query analyzes sales performance for Electronics and Accessories by summarizing transactions per employee, store, and customer loyalty status, while also computing total revenue per store-category pair for ranking and comparison.


<b>Step 1:</b> Identify slow-running queries:

In [ ]:
# Example: Time a complex query
import time

start_time = time.time()
query = """
WITH StoreSales AS (
   SELECT
       e.store_location,
       e.employee_id,
       e.role,
       p.category,
       c.loyalty_program,
       COUNT(s.transaction_id) AS total_sales,
       SUM(s.quantity) AS total_units_sold,
       SUM(s.total_amount) AS total_revenue
   FROM Sales_Transactions s
   JOIN Employee_Records e ON s.employee_id = e.employee_id
   JOIN Product_Details p ON s.product_id = p.product_id
   JOIN Customer_Demographics c ON s.customer_id = c.customer_id
   WHERE p.category IN ('Electronics', 'Accessories')
   GROUP BY e.store_location, e.employee_id, e.role, p.category, c.loyalty_program
),

StoreRankings AS (
   SELECT
       store_location,
       category,
       SUM(total_revenue) AS store_revenue
   FROM StoreSales
   GROUP BY store_location, category
)

SELECT
   ss.store_location,
   ss.employee_id,
   ss.role,
   ss.category,
   ss.loyalty_program,
   ss.total_sales,
   ss.total_units_sold,
   ss.total_revenue
FROM StoreSales ss
JOIN StoreRankings sr ON ss.store_location = sr.store_location AND ss.category = sr.category
ORDER BY ss.store_location, ss.total_revenue DESC;

"""


df = pd.read_sql_query(query, conn)
end_time = time.time()
print(f"Query execution time: {end_time - start_time} seconds")

<b>Step 2:</b> Add an index to optimize your query:

In [ ]:
# Create an index
cursor.execute("CREATE INDEX idx_product_category ON Product_Details(category)")
conn.commit()

<b>Step 3:</b> Use an AI to further optimize your query:

Using an AI of your choice, further optimize your query then paste your updated query into the cell in Step 4.


**Step 4: Try it yourself:** Re-run your query and compare execution time

In [ ]:
# Your Turn: Run your optimized query and compare execution time

start_time = time.time()
query = """
<YOUR CODE HERE>
"""

df = pd.read_sql_query(query, conn)
end_time = time.time()
print(f"Query execution time: {end_time - start_time} seconds")

<b>💡 Tip:</b> Indexes can significantly improve query performance, but they also have overhead. Use them judiciously.

### Activity 4: Generating the Analysis Report  

Now that we've performed our analysis, it's time to compile our findings into a comprehensive report.

<b>Step 1:</b> Summarize key findings:
- List the top 3 insights from your analysis
- Provide supporting data for each insight

<b>Step 2:</b> Include SQL queries:
- For each key insight, provide the SQL query used

<b>Step 3:</b> Document your process:
- Explain your data cleaning steps
- Describe any challenges you encountered and how you overcame them
- Discuss potential areas for further analysis

#### Close the Connection
It's good practice to close the database connection when you're done

In [ ]:
# Close the database connection 
conn.close()

## ✅ Success Checklist
- Cleaned and prepared all dataset tables
- Performed at least 3 advanced SQL queries using subqueries, CTEs, or window functions
- Optimized at least one complex query for better performance
- Compiled a comprehensive analysis report with key insights and supporting data
- Program runs without errors

## 🔍 Common Issues & Solutions 

- Problem: Query returns no results 
    - Solution: Double-check table and column names, and ensure your JOIN conditions are correct
- Problem: Error "no such table" 
     - Solution: Verify that you're connected to the correct database and that the table name is spelled correctly

## ➡️ Summary
In this comprehensive lab, you've applied advanced SQL concepts to analyze TechMart's retail operations data, working with multiple related tables covering employee records, product details, customer demographics, and sales transactions. You've gained hands-on experience in data cleaning, writing complex queries using subqueries, CTEs, and window functions, and optimizing query performance through indexing. Through this real-world simulation, you've developed the practical skills needed to conduct thorough data analysis and create well-documented reports, preparing you for actual data science roles.


### 🔑 Key Points
- Data cleaning is crucial for accurate analysis
- Advanced SQL techniques like CTEs and window functions enable complex analysis
- Query optimization is essential for working with large datasets
- Effective reporting is key to communicating insights